# Machine Learning with Spark

## Introduction

You've now explored how to perform operations on Spark RDDs for simple Map-Reduce tasks. Luckily, there are far more advanced use cases for spark, and many of them are found in the ml library, which we are going to explore in this lesson.


## Objectives
* Describe the use case for Machine Learning with Spark
* Load and manipulate data with Spark DataFrames
* Train a machine learning model with Spark


## A Tale of Two Libraries

If you look at the pyspark documentation, you'll notice that there are two different libraries for machine learning, [mllib](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html) and [ml](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html). These libraries are extremely similar to one another, the only difference being that the mllib library is built upon the RDDs you just practiced using; whereas, the ml library is built on higher level Spark DataFrames, which has methods and attributes similar to pandas. Spark has stated that in the future, it is going to devote more effort to the `ml` library and that `mllib` will become deprecated. It's important to note that these libraries are much younger than pandas and sci-kit learn and there are not as many features present in either.

## Spark DataFrames

In the previous lessons, you've been introduced to SparkContext as the primary way to connect with a Spark Application. Here, we will be using SparkSession, which is from the [sql](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html) component of pyspark. The SparkSession acts the same way as SparkContext; it a bridge between python and the Spark Application. It's just built on top of the Spark SQL API, a higher-level API than RDDs. In fact, a SparkContext object is spun up around which the SparkSession object is wrapped. Let's go through the process of manipulating some data here. For this example, we're going to be using the [Forest Fire dataset](https://archive.ics.uci.edu/ml/datasets/Forest+Fires) from UCI, which contains data about the area burned by wildfires in the Northeast region of Portugal in relation to numerous other factors.

To begin with, let's create a SparkSession so that we can spin up our spark application.


In [1]:
# importing the necessary libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
# sc = SparkContext('local[*]')
# spark = SparkSession(sc)

An alternate, one liner way to create a SparkSession is below

In [2]:
spark = SparkSession.builder.master("local").getOrCreate()

Now, we'll load the read in our data into the pyspark DataFrame object.

In [3]:
## reading in pyspark df
spark_df = spark.read.csv('./forestfires.csv',header='true',inferSchema='true')

## observing the datatype of df
type(spark_df)

pyspark.sql.dataframe.DataFrame

You'll notice that some of the methods are extremely similar or the same as those found within Pandas.

In [4]:
spark_df.head()

Row(X=7, Y=5, month='mar', day='fri', FFMC=86.2, DMC=26.2, DC=94.3, ISI=5.1, temp=8.2, RH=51, wind=6.7, rain=0.0, area=0.0)

In [5]:
spark_df.columns

['X',
 'Y',
 'month',
 'day',
 'FFMC',
 'DMC',
 'DC',
 'ISI',
 'temp',
 'RH',
 'wind',
 'rain',
 'area']

Selecting multiple columns is the same

In [6]:
spark_df[['month','day','rain']]

DataFrame[month: string, day: string, rain: double]

But selecting one column is different. If you want to maintain the methods of a spark DataFrame, you should use the `select` method. If you want to just select the column, you can use the same method you would use in pandas (this is primarily what you would use if you're attempting to create a Boolean mask).


In [7]:
d = spark_df.select('rain')

In [8]:
spark_df['rain']

Column<b'rain'>

Let's take a look at all of our data types in this dataframe

In [9]:
spark_df.dtypes

[('X', 'int'),
 ('Y', 'int'),
 ('month', 'string'),
 ('day', 'string'),
 ('FFMC', 'double'),
 ('DMC', 'double'),
 ('DC', 'double'),
 ('ISI', 'double'),
 ('temp', 'double'),
 ('RH', 'int'),
 ('wind', 'double'),
 ('rain', 'double'),
 ('area', 'double')]

## Aggregations with our DataFrame

Let's investigate to see if there is any correlation between what month it is and the area of fire.

In [10]:
spark_df_months = spark_df.groupBy('month').agg({'area':'mean'})
spark_df_months

DataFrame[month: string, avg(area): double]

Notice how the grouped DataFrame is not returned when you call the aggregation method. Remember, this is still Spark! The transformations and actions are kept separate so that it is easier to manage large quantities of data. You can perform the transformation by making a `collect` method call.

In [11]:
spark_df_months.collect()

[Row(month='jun', avg(area)=5.841176470588234),
 Row(month='aug', avg(area)=12.489076086956521),
 Row(month='may', avg(area)=19.24),
 Row(month='feb', avg(area)=6.275),
 Row(month='sep', avg(area)=17.942616279069753),
 Row(month='mar', avg(area)=4.356666666666667),
 Row(month='oct', avg(area)=6.638),
 Row(month='jul', avg(area)=14.3696875),
 Row(month='nov', avg(area)=0.0),
 Row(month='apr', avg(area)=8.891111111111112),
 Row(month='dec', avg(area)=13.33),
 Row(month='jan', avg(area)=0.0)]

As you can see, there seem to be larger area fires during what would be considered the summer months in Portugal. On your own, practice more aggregations and manipulations that you might be able to perform on this dataset. Now, we'll move on to using the machine learning applications of pyspark. 

## Boolean Masking
Boolean masking also works with pyspark DataFrames just like Pandas DataFrames, the only difference being that the `filter` method is used in pyspark. To try this out, let's compare the amount the fire in those areas with absolutely no rain to those areas that had rain.

In [12]:
no_rain = spark_df.filter(spark_df['rain'] == 0.0)
some_rain = spark_df.filter(spark_df['rain'] > 0.0)

Now, to perform calculations to find the mean of a column, we'll have to import functions from `pyspark.sql`. As always, to read more about them, check out the [documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions).

In [13]:
from pyspark.sql.functions import mean

print('no rain fire area: ',no_rain.select(mean('area')).show(),'\n')

print('some rain fire area: ',some_rain.select(mean('area')).show(),'\n')

+------------------+
|         avg(area)|
+------------------+
|13.023693516699408|
+------------------+

no rain fire area:  None 

+---------+
|avg(area)|
+---------+
|  1.62375|
+---------+

some rain fire area:  None 



Yes there's definitely something there! Unsurprisingly, rain plays in a big factor in the spread of wildfire.

Let's try and get obtain data from only the summer months in Portugal (June, July, and August). We can also do the same for the winter months in Portugal (December, January, February).

In [14]:
summer_months = spark_df.filter(spark_df['month'].isin(['jun','jul','aug']))
winter_months = spark_df.filter(spark_df['month'].isin(['dec','jan','feb']))

print('summer months fire area', summer_months.select(mean('area')).show())
print('winter months fire areas', winter_months.select(mean('area')).show())

+------------------+
|         avg(area)|
+------------------+
|12.262317596566525|
+------------------+

summer months fire area None
+-----------------+
|        avg(area)|
+-----------------+
|7.918387096774193|
+-----------------+

winter months fire areas None


## Machine Learning

Now that we've performed some data manipulation and aggregation, lets get to the really cool stuff, machine learning! Pyspark states that they've used sklearn as an inspiration for their implementation of a machine learning library. As a result, many of the methods and functionalities look similar, but there are some crucial distinctions. There are three main concepts found within the ML library:

`Transformer`: An algorithm that transforms one pyspark DataFrame into another DataFrame. 

`Estimator`: An algorithm that can be fit onto a pyspark DataFrame that can then be used as a Transformer. 

`Pipeline`: A pipeline very similar to an sklearn pipeline that chains together different actions.

The reasoning behind this separation of the fitting and transforming step is because spark is lazily evaluated, so the 'fitting' of a model does not actually take place until the Transformation action is called. Let's examine what this actually looks like by performing a regression on the Forest Fire Dataset. To start off with, we'll import the necessary libraries for our tasks.

In [15]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import feature
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator

Looking at our data, one can see that all the categories are numerical except for day and month. We saw some correlation between the month and area burned in a fire, so we will include that in our model. The day of the week, however, is highly unlikely to have any effect on fire, so we will drop it from the DataFrame.

In [16]:
fire_df = spark_df.drop('day')
fire_df.head()

Row(X=7, Y=5, month='mar', FFMC=86.2, DMC=26.2, DC=94.3, ISI=5.1, temp=8.2, RH=51, wind=6.7, rain=0.0, area=0.0)

In order for us to run our model, we need to turn the months variable into a dummy variable. In `ml` this is a 2-step process that first requires turning the categorical variable into a numerical index (`StringIndexer`). Only after the variable is an int can pyspark create dummy variable columns related to each category (`OneHotEncoderEstimator`). You key parameters you when using these `ml` estimators are: inputCol (the column you want to change) and outputCol (where you will store the changed column). Here it is in action

In [17]:
si = StringIndexer(inputCol='month',outputCol='month_num')
model = si.fit(fire_df)
new_df = model.transform(fire_df)

Note the small, but critical distinction between sklearn's implementation of a transformer and pyspark's implementation. sklearn is more object oriented and spark is more functionally based programming.

In [18]:
## this is an estimator (an untrained transformer)
type(si)

pyspark.ml.feature.StringIndexer

In [19]:
## this is a transformer (a trained transformer)
type(model)

pyspark.ml.feature.StringIndexerModel

In [20]:
model.labels

['aug',
 'sep',
 'mar',
 'jul',
 'feb',
 'jun',
 'oct',
 'apr',
 'dec',
 'jan',
 'may',
 'nov']

In [21]:
new_df.head(4)

[Row(X=7, Y=5, month='mar', FFMC=86.2, DMC=26.2, DC=94.3, ISI=5.1, temp=8.2, RH=51, wind=6.7, rain=0.0, area=0.0, month_num=2.0),
 Row(X=7, Y=4, month='oct', FFMC=90.6, DMC=35.4, DC=669.1, ISI=6.7, temp=18.0, RH=33, wind=0.9, rain=0.0, area=0.0, month_num=6.0),
 Row(X=7, Y=4, month='oct', FFMC=90.6, DMC=43.7, DC=686.9, ISI=6.7, temp=14.6, RH=33, wind=1.3, rain=0.0, area=0.0, month_num=6.0),
 Row(X=8, Y=6, month='mar', FFMC=91.7, DMC=33.3, DC=77.5, ISI=9.0, temp=8.3, RH=97, wind=4.0, rain=0.2, area=0.0, month_num=2.0)]

In [22]:
#
# new_df = new_df.drop('month')
new_df.head()

Row(X=7, Y=5, month='mar', FFMC=86.2, DMC=26.2, DC=94.3, ISI=5.1, temp=8.2, RH=51, wind=6.7, rain=0.0, area=0.0, month_num=2.0)

As you can see, we have created a new column called "month_num" that represents the month by a number. Now that we have performed this step, we can use Spark's version of OneHotEncoder. Let's make sure we have an accurate representation of the months.

In [23]:
new_df.select('month_num').distinct().collect()

[Row(month_num=8.0),
 Row(month_num=0.0),
 Row(month_num=7.0),
 Row(month_num=1.0),
 Row(month_num=4.0),
 Row(month_num=11.0),
 Row(month_num=3.0),
 Row(month_num=2.0),
 Row(month_num=10.0),
 Row(month_num=6.0),
 Row(month_num=5.0),
 Row(month_num=9.0)]

In [24]:
## fitting and transforming the OneHotEncoder
ohe = feature.OneHotEncoderEstimator(inputCols=['month_num'],outputCols=['month_vec'],dropLast=True)
one_hot_encoded = ohe.fit(new_df).transform(new_df)
one_hot_encoded.head()

Row(X=7, Y=5, month='mar', FFMC=86.2, DMC=26.2, DC=94.3, ISI=5.1, temp=8.2, RH=51, wind=6.7, rain=0.0, area=0.0, month_num=2.0, month_vec=SparseVector(11, {2: 1.0}))

Great, we now have a OneHotEncoded sparse vector in the month_vec column! Because spark is optimized for big data, sparse vectors are used rather than entirely new columns for dummy variables because it is more space efficient. You can see in this first row of the data frame:  
`month_vec=SparseVector(11, {2: 1.0})` this indicates that we have a sparse vector of size 11 (because of the parameter `dropLast = True` in OneHotEncoderEstimator) and this particular datapoint is the 2nd index of our month labels (march, based off the labels in the `model` StringEstimator transformer)  

The final requirement for all machine learning models in pyspark is to put all of the features of your model into one Sparse Vector. This is once again for efficiency sake. Here, we are doing that with the `VectorAssembler` estimator.

In [25]:
features = ['X',
 'Y',
 'FFMC',
 'DMC',
 'DC',
 'ISI',
 'temp',
 'RH',
 'wind',
 'rain',
 'month_vec']

target = 'area'

vector = VectorAssembler(inputCols=features,outputCol='features')
vectorized_df = vector.transform(one_hot_encoded)

In [26]:
vectorized_df.head()

Row(X=7, Y=5, month='mar', FFMC=86.2, DMC=26.2, DC=94.3, ISI=5.1, temp=8.2, RH=51, wind=6.7, rain=0.0, area=0.0, month_num=2.0, month_vec=SparseVector(11, {2: 1.0}), features=SparseVector(21, {0: 7.0, 1: 5.0, 2: 86.2, 3: 26.2, 4: 94.3, 5: 5.1, 6: 8.2, 7: 51.0, 8: 6.7, 12: 1.0}))

Great! We now have our data in a format that seems acceptable for the last step. Now it's time for us to actually fit our model to data! Let's try and fit a Random Forest Regression model our data. Although there are still a bunch of other features in the DataFrame, it doesn't matter for the machine learning model API. All that needs to be specified are the names of the features column and the label column. Let's fit use Random Forest Regression to fit the model here.

In [27]:
## instantiating and fitting the model
rf_model = RandomForestRegressor(featuresCol='features',labelCol='area',predictionCol="prediction").fit(vectorized_df)

In [28]:
rf_model.featureImportances

SparseVector(21, {0: 0.1068, 1: 0.0655, 2: 0.1444, 3: 0.1413, 4: 0.1178, 5: 0.0489, 6: 0.0984, 7: 0.0804, 8: 0.1497, 10: 0.0027, 11: 0.0391, 12: 0.0, 13: 0.0042, 14: 0.0002, 16: 0.0, 17: 0.0, 18: 0.0001, 20: 0.0005})

In [29]:
## generating predictions
predictions = rf_model.transform(vectorized_df).select("area","prediction")
predictions.head(10)

[Row(area=0.0, prediction=5.590453951687039),
 Row(area=0.0, prediction=6.0778621166987605),
 Row(area=0.0, prediction=8.559628119674656),
 Row(area=0.0, prediction=7.971877982911005),
 Row(area=0.0, prediction=8.296784649063989),
 Row(area=0.0, prediction=6.71728529187496),
 Row(area=0.0, prediction=3.866013114636226),
 Row(area=0.0, prediction=5.5254393125674985),
 Row(area=0.0, prediction=6.425106730602695),
 Row(area=0.0, prediction=6.353045545022376)]

Now we can evaluate how well the model performed using `RegressionEvaluator`.

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='area')

In [31]:
## evaluating r^2
evaluator.evaluate(predictions,{evaluator.metricName:"r2"})

0.7079057144952472

In [32]:
## evaluating mean absolute error
evaluator.evaluate(predictions,{evaluator.metricName:"mae"})

12.641137683177428

## Putting it all in a Pipeline

We just performed a whole lot of transformations to our data. Let's take a look at all the estimators we used to create this model:

* StringIndexer
* OneHotEnconderEstimator
* VectorAssembler
* RandomForestRegressor

Once we've fit our model in the Pipeline, we're then going to want to evaluate it to determine how well it performs. We can do this with:

* RegressionEvaluator


We can streamline all of these transformations to make it all much more efficient by chaining them together in a pipeline. The Pipeline object expects a list of the estimators prior set to the parameter `stages`.

In [33]:
# importing relevant libraries
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml import Pipeline

In [34]:
## instantiating all necessary estimator objects

string_indexer = StringIndexer(inputCol='month',outputCol='month_num',handleInvalid='keep')
one_hot_encoder = OneHotEncoderEstimator(inputCols=['month_num'],outputCols=['month_vec'],dropLast=True)
vector_assember = VectorAssembler(inputCols=features,outputCol='features')
random_forest = RandomForestRegressor(featuresCol='features',labelCol='area')
stages =  [string_indexer, one_hot_encoder, vector_assember,random_forest]

# instantiating the pipeline with all them estimator objects
pipeline = Pipeline(stages=stages)

### Cross Validation
You might have missed a critical step in the Random Forest Regression above; we did not cross validate or perform a train/test split! Now we're going to fix that by performing cross validation and also testing out multiple different combinations of parameters in pyspark's GridSearch equivalent. To begin with, we will create a parameter grid that contains the different parameters we want to use in our model.

In [35]:
# creating parameter grid

params = ParamGridBuilder()\
.addGrid(random_forest.maxDepth, [5,10,15])\
.addGrid(random_forest.numTrees, [20,50,100])\
.build()

Let's take a look at the params variable we just built.

In [36]:
print('total combinations of parameters: ',len(params))

params[0]

total combinations of parameters:  9


{Param(parent='RandomForestRegressor_bbe0ad2fcccd', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestRegressor_bbe0ad2fcccd', name='numTrees', doc='Number of trees to train (>= 1).'): 20}

Now it's time to combine all the steps we've created to work in a single line of code with the CrossValidator estimator.

In [37]:
## instantiating the evaluator by which we will measure our model's performance
reg_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='area',metricName = 'mae')
## instantiating crossvalidator estimator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params,evaluator=reg_evaluator,parallelism=4)

In [38]:
## fitting crossvalidator
cross_validated_model = cv.fit(fire_df)

Now, let's see how well the model performed! Let's take a look at the average performance for each one of our 9 models. It looks like the optimal performance is an MAE around 23. Note that this is worse than our original model, but that's because our original model had substantial data leakage. We didn't do a train-test-split!

In [39]:
cross_validated_model.avgMetrics

[21.278864718183236,
 20.842858659559905,
 21.973919848048528,
 21.582608982786844,
 21.35375412826563,
 22.74676249558219,
 21.648628857735783,
 21.471197411752346,
 22.894746724672533]

Now, let's take a look at the optimal parameters of our best performing model. The cross_validated_model variable is now saved as the best performing model from the grid search just performed. Let's look to see how well the predictions performed. As you can see, this dataset has a large number of areas of "0.0" burned. Perhaps, it would be better to investigate this problem as a classification task.

In [40]:
predictions = cross_validated_model.transform(spark_df)
predictions.select('prediction','area').show(300)

+------------------+-------+
|        prediction|   area|
+------------------+-------+
| 5.540443763685769|    0.0|
| 5.540209008015621|    0.0|
| 5.255107158319282|    0.0|
| 5.340407951045965|    0.0|
| 4.189406666681819|    0.0|
| 9.937355483673775|    0.0|
|15.044732575071738|    0.0|
| 7.838208493451117|    0.0|
| 6.658159304018691|    0.0|
|30.104691804865958|    0.0|
| 6.739413159664569|    0.0|
| 5.819930767884711|    0.0|
| 8.157287420120994|    0.0|
| 8.990780759575582|    0.0|
| 43.24112497400585|    0.0|
| 7.269366355653526|    0.0|
|20.345841415363132|    0.0|
|   7.7524144705357|    0.0|
|4.5179108674772115|    0.0|
| 4.660482384371875|    0.0|
| 8.263065113826098|    0.0|
| 4.963400481173717|    0.0|
| 4.357778867394337|    0.0|
|  5.66686223643433|    0.0|
| 5.532855945461998|    0.0|
|  6.41953882466031|    0.0|
| 7.892709825614431|    0.0|
|  7.50301603419343|    0.0|
|31.108448757810542|    0.0|
|10.055895568029378|    0.0|
| 7.395599462088808|    0.0|
| 7.2016948193

Now let's go ahead and take a look at the feature importances of our Random Forest model. In order to do this, we need to unroll our pipeline to access the Random Forest Model. Let's start by first checking out the "bestModel" attribute of our cross_validated_model.

In [41]:
type(cross_validated_model.bestModel)

pyspark.ml.pipeline.PipelineModel

`ml` is treating the entire pipeline as the best performing model, so we need to go deeper into the pipeline to access the Random Forest model within it. Previously, we put the Random Forest Model as the final "stage" in the stages variable list. Let's look at the stages attribute of the bestModel.

In [42]:
cross_validated_model.bestModel.stages

[StringIndexer_da64abb0e0db,
 OneHotEncoderEstimator_f11ef3ee25de,
 VectorAssembler_3d4a7fae1bc1,
 RandomForestRegressionModel (uid=RandomForestRegressor_bbe0ad2fcccd) with 50 trees]

Perfect! There's the RandomForestRegressionModel, represented by the last item in the stages list. Now, we should be able to access all the attributes of the Random Forest Regressor.

In [43]:
optimal_rf_model = cross_validated_model.bestModel.stages[3]

In [44]:
optimal_rf_model.featureImportances

SparseVector(22, {0: 0.0856, 1: 0.1027, 2: 0.1122, 3: 0.1779, 4: 0.1098, 5: 0.0469, 6: 0.1004, 7: 0.0712, 8: 0.1274, 9: 0.0, 10: 0.0104, 11: 0.0322, 12: 0.0002, 13: 0.0206, 14: 0.0003, 15: 0.0001, 16: 0.0001, 17: 0.0015, 18: 0.0001, 20: 0.0004})

In [45]:
optimal_rf_model.getNumTrees

50

## Summary

In this lesson, you have learned about pyspark's DataFrames, machine learning models, and pipelines. With the use of a pipeline, you can train a huge number of models simultaneously, saving you a substantial amount of time and effort. Up next, you will have a chance to build a pyspark machine learning pipeline of your own with a classification problem!